<a href="https://colab.research.google.com/github/JayThibs/gpt-stackoverflow-QA/blob/main/data_preparation/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing StackOverflow QA Data

This notebook creates a train, validation, and test sets containing a StackOverflow question with its corresponding top answer.

In [1]:
!nvidia-smi

Thu Dec  9 00:43:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installations

In [5]:
!pip install xmltodict wget pyunpack patool --quiet

     |████████████████████████████████| 77 kB 2.8 MB/s 


# Imports

In [127]:
import os
import re
import wget
from pyunpack import Archive
import xmltodict
import xml.etree.ElementTree as ET
from collections import OrderedDict
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [8]:
np.random.seed(3407)

# Getting the Data

In [9]:
url = 'https://archive.org/download/stackexchange/ai.stackexchange.com.7z'
wget.download(url)

'ai.stackexchange.com.7z'

In [51]:
if not os.path.isdir('./data'):
    os.mkdir('./data')
if len(os.listdir('./data') ) == 0:
    Archive('ai.stackexchange.com.7z').extractall("./data")

In [27]:
tree = ET.parse('./data/Posts.xml')
tree = tree.getroot()

In [37]:
xmlstr = ET.tostring(tree, encoding='utf8', method='xml').decode()

In [38]:
soDict = xmltodict.parse(xmlstr)

In [133]:
type(soDict)

collections.OrderedDict

In [134]:
soDict['posts']['row'][0]

OrderedDict([('@AcceptedAnswerId', '3'),
             ('@AnswerCount', '5'),
             ('@Body',
              '<p>What does "backprop" mean? Is the "backprop" term basically the same as "backpropagation" or does it have a different meaning?</p>\n'),
             ('@CommentCount', '0'),
             ('@ContentLicense', 'CC BY-SA 4.0'),
             ('@CreationDate', '2016-08-02T15:39:14.947'),
             ('@FavoriteCount', '1'),
             ('@Id', '1'),
             ('@LastActivityDate', '2021-07-08T10:45:23.250'),
             ('@LastEditDate', '2019-11-16T17:56:22.093'),
             ('@LastEditorUserId', '2444'),
             ('@OwnerUserId', '8'),
             ('@PostTypeId', '1'),
             ('@Score', '10'),
             ('@Tags',
              '<neural-networks><backpropagation><terminology><definitions>'),
             ('@Title', 'What is "backprop"?'),
             ('@ViewCount', '625')])

In [66]:
soDict['posts']['row'][2]

OrderedDict([('@Body',
              '<p>"Backprop" is the same as "backpropagation": it\'s just a shorter way to say it. It is sometimes abbreviated as "BP".</p>\n'),
             ('@CommentCount', '0'),
             ('@ContentLicense', 'CC BY-SA 3.0'),
             ('@CreationDate', '2016-08-02T15:40:24.820'),
             ('@Id', '3'),
             ('@LastActivityDate', '2016-08-02T15:40:24.820'),
             ('@OwnerUserId', '4'),
             ('@ParentId', '1'),
             ('@PostTypeId', '2'),
             ('@Score', '15')])

In [59]:
len(soDict['posts']['row'])

21482

In [72]:
soQADict = {}

In [108]:
counter = 0

for i in range(len(soDict['posts']['row'])):
    post = soDict['posts']['row'][i]
    if post.__contains__('@AcceptedAnswerId'):
        post_id = counter
        counter += 1
        AcceptedAnswerId = int(post['@AcceptedAnswerId'])
        soQADict[post_id] = {}
        try:
            soQADict[post_id]['BestAnswer'] = soDict['posts']['row'][AcceptedAnswerId]['@Body']
            soQADict[post_id]['AnswerScore'] = soDict['posts']['row'][AcceptedAnswerId]['@Score']
            soQADict[post_id]['AnswerCreationDate'] = soDict['posts']['row'][AcceptedAnswerId]['@CreationDate']
        except (IndexError):
            continue
        soQADict[post_id]['Title'] = post['@Title']
        soQADict[post_id]['Question'] = post['@Body']
        soQADict[post_id]['QuestionScore'] = post['@Score']
        soQADict[post_id]['QuestionCreationDate'] = post['@CreationDate']
        soQADict[post_id]['QuestionTags'] = post['@Tags']
        if post.__contains__('@FavoriteCount'):
            soQADict[post_id]['FavoriteCount'] = post['@FavoriteCount']
        soQADict[post_id]['ViewCount'] = post['@ViewCount']

In [110]:
len(soQADict)

3235

In [129]:
soQADict = OrderedDict(soQADict)

In [132]:
soQADict[0]

{'AnswerCreationDate': '2016-08-02T15:41:22.020',
 'AnswerScore': '32',
 'BestAnswer': "<p>When you're writing your algorithm, how do you know how many neurons you need per single layer? Are there any methods for finding the optimal number of them, or is it a rule of thumb?</p>\n",
 'FavoriteCount': '1',
 'Question': '<p>What does "backprop" mean? Is the "backprop" term basically the same as "backpropagation" or does it have a different meaning?</p>\n',
 'QuestionCreationDate': '2016-08-02T15:39:14.947',
 'QuestionScore': '10',
 'QuestionTags': '<neural-networks><backpropagation><terminology><definitions>',
 'Title': 'What is "backprop"?',
 'ViewCount': '625'}

In [131]:
from itertools import islice
list(islice(sorted(soQADict.items()), 5))

TypeError: ignored